In [1]:
import os
import pandas as pd
import vaex
import numpy as np
import pyarrow as pa

### Data from CSV files to analyse

In [ ]:
os.chdir("CSV_files")

In [ ]:
with open("CSV_files_url.txt", 'r') as f:
    files_url = [line.rstrip('\n') for line in f]

In [ ]:
print(f'Number of all CSV files={len(files_url)}')

### We take one of the file and check what kind of data are inside it (column names, data types)

In [ ]:
pd.read_csv(files_url[5])

### In only one file we have got almost 200k of rows, so we can expect 261*200k=50M of rows in all datasets.

### We are interesting only in column `'Start date'` which allow us to calculate number of shared bikes during every hour from all the files

In [ ]:
os.chdir('..') # go to 'London_bike' directory
os.chdir('Parquet_files') # go to 'Parquet_files' directory


def CSV_to_parquet():
    ## Because CSV files on AWS server can not be downloaded simultanously 
    ## (after few minutes connecting was always closed automatically), 
    ## I have to download files one by one (in 'for loop' like that below)
    for j, k in zip(range(len(files_url)), files_url):
        _ = pd.read_csv(k, usecols=['Start Date'])
        _.to_parquet(f'datetime_part_{j}.parquet')


CSV_to_parquet()

In [2]:
import glob

df = vaex.open('datetime_part_*.parquet')
df.info()

### As I said before, this is dataframe with ~50 million rows.

In [3]:
import datetime as dt

def convert_to_datetime(date_string):
    return np.datetime64(dt.datetime.strptime(str(date_string), "%d/%m/%Y %H:%M"))

def Date_hour_split(df):
    _ = df['Start Date'].apply(convert_to_datetime)
    df['Dates'] = _.dt.date
    df['Hour'] = _.dt.hour
    #
    return df.drop(['Start Date'])

split_df = Date_hour_split(df)

In [4]:
split_df

#,Dates,Hour
0,2018-03-10,12
1,2018-03-07,15
2,2018-03-08,13
3,2018-03-09,8
4,2018-03-13,6
...,...,...
"52,188,432",2020-02-03,7
"52,188,433",2020-01-31,19
"52,188,434",2020-01-29,19
"52,188,435",2020-02-03,8


In [24]:
split_df.groupby(['Dates','Hour'], agg='sum')

#,Dates,Hour
0,2020-08-05,0
1,2020-08-05,1
2,2020-08-05,2
3,2020-08-05,3
4,2020-08-05,4
...,...,...
"43,836",2019-09-16,19
"43,837",2019-09-16,20
"43,838",2019-09-16,21
"43,839",2019-09-16,22


### Because Vaex has problem with dropping duplicate values after grouping data, I will use once again Pandas Dataframe.

In [5]:
df = split_df.to_pandas_df()
df

,Dates,Hour
0,2018-03-10,12
1,2018-03-07,15
2,2018-03-08,13
3,2018-03-09,8
4,2018-03-13,6
...,...,...
52188432,2020-02-03,7
52188433,2020-01-31,19
52188434,2020-01-29,19
52188435,2020-02-03,8


In [32]:
def Bikes_share_add(df):
    ## Inserting column without data
    df['Sum'] = ''
    ## Sum number of bikes sharing for every hour and date
    df = df.groupby(['Dates','Hour'], as_index=False)['Sum'].size()
    df = df.rename(columns={'size': 'Bikes share'})
    return df

df_bikeshare = Bikes_share_add(df)

In [33]:
df_bikeshare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43841 entries, 0 to 43840
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Dates        43841 non-null  datetime64[ns]
 1   Hour         43841 non-null  int64         
 2   Bikes share  43841 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.0 MB


In [34]:
df_bikeshare.head()

,Dates,Hour,Bikes share
0,2016-12-28,0,102
1,2016-12-28,1,75
2,2016-12-28,2,30
3,2016-12-28,3,28
4,2016-12-28,4,32


In [37]:
df_bikeshare = df_bikeshare[(df_bikeshare['Dates'] >= '2017-01-01') & (df_bikeshare['Dates'] <= '2022-01-01')].reset_index(drop=True)
df_bikeshare.head()

,Dates,Hour,Bikes share
0,2017-01-01,0,966
1,2017-01-01,1,910
2,2017-01-01,2,346
3,2017-01-01,3,286
4,2017-01-01,4,189


### Save final result as .parquet file

In [38]:
df_bikeshare.to_parquet('df_bikeshares.parquet', compression='snappy')